In [1]:
using CUDA

In [2]:
CUDA.versioninfo()

CUDA toolkit 10.2.89, artifact installation
CUDA driver 10.2.0
NVIDIA driver 430.26.0

Libraries: 


┌ Warning: Your Tesla K20m GPU does not meet the minimal required compute capability (3.5.0 < 5.0).
│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
└ @ CUDA /homen1/mario_aor/.julia/packages/CUDA/wTQsK/src/state.jl:251


- CUBLAS: 10.2.2
- CURAND: 10.1.2
- CUFFT: 10.1.2
- CUSOLVER: 10.3.0
- CUSPARSE: 10.3.1
- CUPTI: 12.0.0
- NVML: 10.0.0+430.26
- CUDNN: 8.0.4 (for CUDA 10.2.0)
- CUTENSOR: 1.2.1 (for CUDA 10.2.0)

Toolchain:
- Julia: 1.5.3
- LLVM: 9.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4
- Device support: sm_30, sm_32, sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75

1 device:
  0: Tesla K20m (sm_35, 4.534 GiB / 4.633 GiB available)


In [3]:
[CUDA.capability(dev) for dev in CUDA.devices()]

1-element Array{VersionNumber,1}:
 v"3.5.0"

In [4]:
@assert CUDA.functional(true)

CUDA.jl propone el uso de tres capas de abstracción:
 - CuArray, que es un tipo de arreglos que implícitamente utiliza el GPU.
 - Programación de kernels con @cuda, en caso de tener cuellos de botella con los CuArrays
 - CUDA API wrappers, que permiten la utilización de características avanzadas de CUDA.

In [5]:
a = CUDA.ones(1024) * 5
b = CUDA.ones(1024)

1024-element CuArray{Float32,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [6]:
c = a + b

1024-element CuArray{Float32,1}:
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 ⋮
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0

In [7]:
c .+ 3

1024-element CuArray{Float32,1}:
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 ⋮
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0
 9.0

In [8]:
c

1024-element CuArray{Float32,1}:
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 ⋮
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0
 6.0

In [9]:
arr1 = collect(1:1024)

1024-element Array{Int64,1}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
   11
   12
   13
    ⋮
 1013
 1014
 1015
 1016
 1017
 1018
 1019
 1020
 1021
 1022
 1023
 1024

In [10]:
c .+ arr1

LoadError: GPU compilation of kernel broadcast_kernel(CUDA.CuKernelContext, CuDeviceArray{Float32,1,1}, Base.Broadcast.Broadcasted{Nothing,Tuple{Base.OneTo{Int64}},typeof(+),Tuple{Base.Broadcast.Extruded{CuDeviceArray{Float32,1,1},Tuple{Bool},Tuple{Int64}},Base.Broadcast.Extruded{Array{Int64,1},Tuple{Bool},Tuple{Int64}}}}, Int64) failed
KernelError: passing and using non-bitstype argument

Argument 4 to your kernel function is of type Base.Broadcast.Broadcasted{Nothing,Tuple{Base.OneTo{Int64}},typeof(+),Tuple{Base.Broadcast.Extruded{CuDeviceArray{Float32,1,1},Tuple{Bool},Tuple{Int64}},Base.Broadcast.Extruded{Array{Int64,1},Tuple{Bool},Tuple{Int64}}}}, which is not isbits:
  .args is of type Tuple{Base.Broadcast.Extruded{CuDeviceArray{Float32,1,1},Tuple{Bool},Tuple{Int64}},Base.Broadcast.Extruded{Array{Int64,1},Tuple{Bool},Tuple{Int64}}} which is not isbits.
    .2 is of type Base.Broadcast.Extruded{Array{Int64,1},Tuple{Bool},Tuple{Int64}} which is not isbits.
      .x is of type Array{

In [11]:
gpuarr1 = CUDA.CuArray(arr1)

1024-element CuArray{Int64,1}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
   11
   12
   13
    ⋮
 1013
 1014
 1015
 1016
 1017
 1018
 1019
 1020
 1021
 1022
 1023
 1024

In [12]:
gpuarr1 = CUDA.CuArray{Float32}(arr1)

1024-element CuArray{Float32,1}:
    1.0
    2.0
    3.0
    4.0
    5.0
    6.0
    7.0
    8.0
    9.0
   10.0
   11.0
   12.0
   13.0
    ⋮
 1013.0
 1014.0
 1015.0
 1016.0
 1017.0
 1018.0
 1019.0
 1020.0
 1021.0
 1022.0
 1023.0
 1024.0

In [13]:
gpuarr2 = c + gpuarr1

1024-element CuArray{Float32,1}:
    7.0
    8.0
    9.0
   10.0
   11.0
   12.0
   13.0
   14.0
   15.0
   16.0
   17.0
   18.0
   19.0
    ⋮
 1019.0
 1020.0
 1021.0
 1022.0
 1023.0
 1024.0
 1025.0
 1026.0
 1027.0
 1028.0
 1029.0
 1030.0

In [14]:
log.(gpuarr1)

1024-element CuArray{Float32,1}:
 0.0
 0.6931472
 1.0986123
 1.3862944
 1.609438
 1.7917595
 1.9459102
 2.0794415
 2.1972246
 2.3025851
 2.3978953
 2.4849067
 2.5649493
 ⋮
 6.9206715
 6.921658
 6.922644
 6.923629
 6.9246125
 6.9255953
 6.926577
 6.927558
 6.928538
 6.929517
 6.930495
 6.931472

In [15]:
map(log, gpuarr1)

1024-element CuArray{Float32,1}:
 0.0
 0.6931472
 1.0986123
 1.3862944
 1.609438
 1.7917595
 1.9459102
 2.0794415
 2.1972246
 2.3025851
 2.3978953
 2.4849067
 2.5649493
 ⋮
 6.9206715
 6.921658
 6.922644
 6.923629
 6.9246125
 6.9255953
 6.926577
 6.927558
 6.928538
 6.929517
 6.930495
 6.931472

In [34]:
CUDA.memory_status()

Effective GPU memory usage: 3.15% (149.250 MiB/4.633 GiB)
CUDA allocator usage: 48.043 MiB
binned usage: 48.043 MiB (32.043 MiB allocated, 16.000 MiB cached)


In [35]:
a = CuArray{Int}(undef, 2000000);

In [36]:
CUDA.memory_status()

Effective GPU memory usage: 3.15% (149.250 MiB/4.633 GiB)
CUDA allocator usage: 48.043 MiB
binned usage: 48.043 MiB (48.043 MiB allocated, 0 bytes cached)


In [37]:
a = nothing

In [38]:
CUDA.memory_status()

Effective GPU memory usage: 3.15% (149.250 MiB/4.633 GiB)
CUDA allocator usage: 48.043 MiB
binned usage: 48.043 MiB (48.043 MiB allocated, 0 bytes cached)


In [39]:
a = CuArray{Int}(undef, 2000000);

In [46]:
CUDA.unsafe_free!(a)

In [50]:
CUDA.memory_status()

Effective GPU memory usage: 2.13% (101.250 MiB/4.633 GiB)
CUDA allocator usage: 44.000 KiB
binned usage: 44.000 KiB (44.000 KiB allocated, 0 bytes cached)
